# Третье домашнее задание
https://docs.google.com/document/d/1KYoqG6dbfzcRCEXYWS1iKxQR9hZEHQUThW4moXWwvhk

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style("whitegrid")
figsize = (12,6)

pd.set_option('display.max_columns', 30)

## Загружаем данные

In [2]:
tournaments = pickle.load(open('data/tournaments.pkl', 'rb'))
results = pickle.load(open('data/results.pkl', 'rb'))
players = pickle.load(open('data/players.pkl', 'rb'))

In [3]:
def parse_tournaments(tournaments, year):
    records = []
    for t in tournaments.values():
        if year is not None and t['dateStart'][:4] != year:
            continue
        records.append({
            'id': t['id'],
            'name': t['name'],
            'dateStart': t['dateStart'],
            'questionQty': sum(t['questionQty'].values())
        })
    return pd.DataFrame.from_records(records)

In [4]:
train_tournaments = parse_tournaments(tournaments, '2019')
test_tournaments = parse_tournaments(tournaments, '2020')

In [5]:
print(train_tournaments.shape)
train_tournaments.head()

(687, 4)


,dateStart,id,name,questionQty
0,2019-01-05T19:00:00+03:00,4772,Синхрон северных стран. Зимний выпуск,36
1,2019-01-25T19:05:00+03:00,4973,Балтийский Берег. 3 игра,36
2,2019-03-01T19:05:00+03:00,4974,Балтийский Берег. 4 игра,36
3,2019-04-05T19:05:00+03:00,4975,Балтийский Берег. 5 игра,36
4,2019-02-15T20:00:00+03:00,4986,ОВСЧ. 6 этап,36


In [6]:
print(test_tournaments.shape)
test_tournaments.head()

(418, 4)


,dateStart,id,name,questionQty
0,2020-12-30T16:00:00+03:00,4628,Семь сорок,36
1,2020-02-21T00:00:00+03:00,4957,Синхрон Биркиркары,39
2,2020-08-01T14:00:00+03:00,5151,Яровой,36
3,2020-01-03T19:00:00+03:00,5414,Синхрон северных стран,36
4,2020-04-18T19:00:00+03:00,5477,Онлайн: Синхрон Урюбджирова,36


In [7]:
mask_translator = str.maketrans('-?', '00', 'X')

def fix_mask(mask):
    """Удаляет снятые вопросы (X) и заменяет не взятые никем вопросы (-) на 0"""
    return mask.translate(mask_translator) if mask else None

In [8]:
fix_mask('00X01-1?')

'0001010'

In [9]:
def parse_results(results, ids):
    records = []
    for id in ids:
        result = results[id]
        for team in result:
            if 'questionsTotal' not in team:
                continue
            mask = fix_mask(team['mask']) if 'mask' in team else None
            for player in team['teamMembers']:
                player = player['player']
                records.append({
                    'tournament_id': id,
                    'team_id': team['team']['id'],
                    'player_id': player['id'],
                    'team_name': team['team']['name'],
                    'teamQuestions': team['questionsTotal'],
                    'player_name': '{} {} {}'.format(player['surname'], player['name'], player['patronymic']),
                    'mask': mask,
                    'mask_len': len(mask or '')
                })
    df = pd.DataFrame.from_records(records)
    df = df.merge(df.groupby('tournament_id')['mask_len'].max().rename('max_mask_len'),
                  left_on='tournament_id', right_index=True)
    # Выбрасываем примеры, в которых нет маски, или её длина не равна максимальной в турнире
    df = df[~df['mask'].isnull() & (df['mask_len'] == df['max_mask_len'])].reset_index(drop=True)
    df.drop(['max_mask_len'], axis=1, inplace=True)
    return df

In [10]:
train_results = parse_results(results, train_tournaments.id)
print(len(train_results))
train_results.head()

414774


,mask,mask_len,player_id,player_name,teamQuestions,team_id,team_name,tournament_id
0,111111111011111110111111111100010010,36,6212,Выменец Юрий Яковлевич,28,45556,Рабочее название,4772
1,111111111011111110111111111100010010,36,18332,Либер Александр Витальевич,28,45556,Рабочее название,4772
2,111111111011111110111111111100010010,36,18036,Левандовский Михаил Ильич,28,45556,Рабочее название,4772
3,111111111011111110111111111100010010,36,22799,Николенко Сергей Игоревич,28,45556,Рабочее название,4772
4,111111111011111110111111111100010010,36,15456,Коновалов Сергей Владимирович,28,45556,Рабочее название,4772


In [11]:
test_results = parse_results(results, test_tournaments.id)
print(len(test_results))
test_results.head()

108394


,mask,mask_len,player_id,player_name,teamQuestions,team_id,team_name,tournament_id
0,11111101111111011011101111000101010001,38,30152,Сорожкин Артём Сергеевич,26,49804,Борский корабел,4957
1,11111101111111011011101111000101010001,38,30270,Спешков Сергей Леонидович,26,49804,Борский корабел,4957
2,11111101111111011011101111000101010001,38,27822,Савченков Михаил Владимирович,26,49804,Борский корабел,4957
3,11111101111111011011101111000101010001,38,28751,Семушин Иван Николаевич,26,49804,Борский корабел,4957
4,11111101111111011011101111000101010001,38,27403,Руссо Максим Михайлович,26,49804,Борский корабел,4957


In [12]:
train_players = set(train_results.player_id)
test_players = set(test_results.player_id)
len(train_players), len(test_players), len(train_players & test_players)

(57424, 28996, 22946)

# Обучаем baseline
Делаем ряд наивных предположений:
- если команда ответила на вопрос, то каждый её участник ответил
- у каждого вопроса своя сложность, и она ни от чего не зависит
- вероятность ответа на вопрос игроком моделируется как логистическая регрессия от двух признаков - идентификатора игрока и идентификатора вопроса, соответственно обучаются сила игрока и сложность вопроса

Замечания:
1. В тестовой выборке есть игроки, которые отсутствовали в обучающей. Для начала можно оценить их силу как среднюю силу по всей обучающей выборке.
2. Конечно же мы не должны знать сложностей вопросов в тестовой выборке. Да это и не нужно: поскольку в нашей модели нет межфакторного взаимодействия между силой игрока и сложностью вопроса, можно подставить среднюю сложность по выборке. Разумеется это не даст корректную вероятность ответа на вопрос, но вполне пригодно для ранжирования.

### Готовим обучающий датасет

In [13]:
class PlayerTokenizer(object):
    """Назначает каждому игроку уникальное натуральное число.
    Числа идут подряд начиная с нуля."""
    def __init__(self):
        self.player2token = {}
        self.token2player = {}
    
    def fit(self, results):
        player_ids = results.player_id.unique()
        self.player2token = {id: i for i, id in enumerate(player_ids)}
        self.token2player = {i: id for i, id in enumerate(player_ids)}
        return self
    
    def get_token(self, player_id):
        return self.player2token[player_id] if player_id in self.player2token else None
    
    def get_player(self, token):
        return self.token2player[token] if token in self.token2player else None

In [14]:
pt = PlayerTokenizer().fit(train_results)

In [15]:
pt.get_token(158635), pt.get_player(pt.get_token(158635))

(9770, 158635)

In [16]:
class QuestionTokenizer(object):
    """Назначает каждому вопросу уникальное натуральное число.
    Числа идут подряд начиная с нуля."""
    def __init__(self):
        self.question2token = {}
        self.token2question = {}
        self.max_token = None
    
    def fit(self, results):
        results = results.drop_duplicates('tournament_id')
        last_id = 0
        for tournament_id, mask_len in zip(results.tournament_id, results.mask_len):
            tournament_tokens = np.arange(last_id, last_id + mask_len)
            for i, tokens in enumerate(tournament_tokens):
                self.token2question[tokens] = (tournament_id, i)
            last_id += mask_len
            self.question2token[tournament_id] = tournament_tokens
        self.max_token = last_id - 1
        return self
    
    def get_token(self, tournament_id, question_index):
        return self.question2token[tournament_id][question_index]
    
    def get_question(self, token):
        return self.token2question[token]

In [17]:
qt = QuestionTokenizer().fit(train_results)

In [18]:
qt.get_token(6255, 214), qt.get_question(qt.get_token(6255, 214))

(33302, (6255, 214))

In [19]:
qt.max_token

33302

In [20]:
from scipy.sparse import csr_matrix
from tqdm import tqdm

def to_csr(X, question_max_token=qt.max_token):
    """Принимает на вход матрицу, имеющую в каждой строке 2 числа - 
    токен вопроса и токен игрока. Возвращает CSR-матрицу,
    строки которой - сконкатенированные one-hot-encoded
    представления вопроса и игрока"""
    # В каждой строке есть ровно две единицы - в позиции текущего вопроса и в позиции игрока
    x_rows = np.repeat(np.arange(X.shape[0]), 2)

    # Номер столбца для первой единицы определяется просто - это токен вопроса
    # Номер второго столбца - токен игрока + сдвиг на максимально возможый токен вопроса
    x_cols = X.copy()
    x_cols[:,1] += question_max_token + 1
    x_cols = x_cols.ravel()
    # Все значения - единицы, т.к. это one-hot encoding
    x_values = np.ones(X.shape[0] * 2, dtype=np.uint8)
    csr_x = csr_matrix((x_values, (x_rows, x_cols)), dtype=np.uint8)
    return csr_x

def make_dataset(results, pt=pt, qt=qt):
    """Генерирует повопросный датасет. Целевые переменные -
    дал игрок корректный ответ (1), или нет (0). Признаки - 
    токен вопроса и токен игрока, закодированные как CSR-матрица
    из сконкатенированных one-hot-представлений токенов"""
    X, y = [], []
    for mask, tid, pid in tqdm(zip(results['mask'], results.tournament_id, results.player_id)):
        player_token = pt.get_token(pid)
        for i, target in enumerate(mask):
            assert target in ['0', '1'], f'Got incorrect mask value: {target} (tournament_id={tid}, player_id={pid})'
            target = int(target)
            question_token = qt.get_token(tid, i)
            X.append([question_token, player_token])
            y.append(target)
    X, y = np.array(X), np.array(y)
    X = to_csr(X, qt.max_token)
    return X, y

In [21]:
train_X, train_y = make_dataset(train_results)

414774it [00:37, 10971.82it/s]


In [22]:
train_X.shape, train_y.shape

((17753544, 90727), (17753544,))

### Обучаем модель

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
model = LogisticRegression(penalty='l2', C=10, solver='liblinear').fit(train_X, train_y)

In [25]:
def get_player_weights(model, qt=qt):
    # В model.coef_ сначала идут обученные сложности вопросов,
    # а начиная с позиции (qt.max_token + 1) - силы игроков
    return model.coef_[0, qt.max_token + 1:]

In [26]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [27]:
def get_player_score(player_id, model, pt=pt, qt=qt):
    """Предсказанная моделью вероятность дачи правильного ответа
    игроком player_id на вопрос сложности 0"""
    player_token = pt.get_token(player_id)
    player_weights = get_player_weights(model, qt)
    # Если в тестовой выборке встретится неизвестный игрок, оценим его силу как среднюю по обученным силам
    mean_player_weight = player_weights.mean()
    player_weight = player_weights[player_token] if player_token is not None else mean_player_weight
    return sigmoid(player_weight + model.intercept_)

In [28]:
def get_team_score(player_ids, model, pt=pt):
    """Вероятность того, что хотя бы один игрок из списка player_ids
    даст верный ответ на вопрос сложности 0"""
    scores = np.array([get_player_score(pid, model, pt) for pid in player_ids])
    return 1 - np.prod(1 - scores)

In [29]:
from scipy.stats import spearmanr, kendalltau

def eval_model(model, results, pt=pt, with_tqdm=True):
    """Считает средние значения Spearman\'s R и Kendall\'s Tau по турнирам"""
    spearmanrs = []
    kendalltaus = []
    iterable = results.groupby('tournament_id')
    if with_tqdm:
        iterable = tqdm(iterable)
    for _, tournament_df in iterable:
        # Если в турнире участвовала всего одна команда, то сравнивать нечего
        if len(tournament_df.team_id.unique()) == 1:
            continue
        true_scores = []
        pred_scores = []
        for _, team_df in tournament_df.groupby('team_id'):
            team_players = team_df.player_id.values
            team_true_score = team_df.teamQuestions.iloc[0]
            team_pred_score = get_team_score(team_players, model, pt)
            true_scores.append(team_true_score)
            pred_scores.append(team_pred_score)
        spearmanrs.append(spearmanr(true_scores, pred_scores).correlation)
        kendalltaus.append(kendalltau(true_scores, pred_scores).correlation)
    return np.mean(spearmanrs), np.mean(kendalltaus)

In [30]:
spearman, kendall = eval_model(model, test_results)
print(f'Spearman\'s R: {spearman}\tKendall\'s Tau: {kendall}')

100%|██████████| 173/173 [00:12<00:00, 13.57it/s]

Spearman's R: 0.7742819352714337	Kendall's Tau: 0.6192701345684648


# Применяем EM-алгоритм
Здесь мы избавляемся от предположения о том, что если команда ответила верно, то каждый её участник также верно ответил. Вместо этого делаем другие предположения:  
1. Если команда не дала верного ответа, то и ни один из её участников не дал.  
2. Если игрок дал верный ответ, то и его команда верно ответила.  

Но возникает трудность: в качестве правильного ответа в обучающей выборке должны быть метки для каждого игрока - дал он верный ответ, или же нет. Вместо этого у нас есть подобные метки для команд. Что же делать? Введём скрытые переменные - ответы игроков, ответы же команд будут наблюдаемыми переменными.  

Обозначения:  
$\tau$ - индекс турнира  
$t$ - индекс команды  
$t \in \tau$ - команда $t$ участвует в турнире $\tau$  
$p \in t$ - игрок $p$ играет за команду $t$  
$q \in \tau$ - вопрос из турнира $\tau$  
$A_t^q \in \{0,1\}$ - ответ команды $t$ на вопрос $q$ ($0$ - неверный ответ, $1$ - верный)  
$a_p^q \in \{0,1\}$ - ответ игрока $p$ на вопрос $q$ ($0$ - неверный ответ, $1$ - верный)  

Предположения:  
$\forall p \in t \quad P(a_p^q=1|A_t^q=0) = 0$  
$\forall p \in t \quad P(A_t^q=1|a_p^q=1) = 1$  

В качестве параметров модели $\theta$ по-прежнему будут выступать силы игроков $w_p$, сложности вопросов $w_q$ и свободный член $b$, сама же модель - снова логистическая регрессия. EM-схема выглядит следующим образом:  
E-шаг:  
- Оцениваем скрытые переменные $a_p^q$ при фиксированных параметрах $\theta^{(m)}:$  
$\operatorname{E}[a_p^q|A_t^q,\theta^{(m)}] = P(a_p^q=1|A_t^q,\theta^{(m)}) = (*)$  
Если $A_t^q=0,$ то $(*) = 0$, иначе  
$$(*) = P(a_p^q=1|A_t^q=1,\theta^{(m)}) = \frac{ P(A_t^q=1|a_p^q=1) P(a_p^q=1 | \theta^{(m)}) }{ P(A_t^q=1|\theta^{(m)}) } = \frac{ P(a_p^q=1 | \theta^{(m)}) }{ 1 - P(A_t^q=0 | \theta^{(m)}) } = \frac{ P(a_p^q=1 | \theta^{(m)}) }{ 1 - \prod_{g \in t}P(a_g^q=0 | \theta^{(m)}) }$$

M-шаг:  
- При фиксированных $a_p^q$ (точнее при их ожиданиях при $\theta^{(m)}$  ) максимизируем правдоподобие $P(A,a|\theta)$ по параметрам $\theta,$ то есть просто обучаем логистическую регрессию на целевые переменные $a_p^q$. В результате оптимизации получаем новые параметры $\theta^{(m+1)}$

In [31]:
def make_prediction_meta(results, pt=pt, qt=qt):
    """В дополнение к матрице признаков и целевому вектору генерирует
    датафрейм, позволяющий восстановить токен вопроса и id команды
    для каждого объекта выборки"""
    q_tokens, team_ids, y = [], [], []
    for mask, tournament_id, team_id in tqdm(zip(results['mask'], results.tournament_id, results.team_id)):
        q_tokens.extend([qt.get_token(tournament_id, i) for i in range(len(mask))])
        team_ids.extend([team_id] * len(mask))  
        y.extend([bool(int(c)) for c in mask])
    return pd.DataFrame({'question_token': q_tokens, 'team_id': team_ids, 'team_answered': y})

In [32]:
train_prediction_meta = make_prediction_meta(train_results)

414774it [00:12, 33289.80it/s]


In [33]:
assert train_y.shape[0] == train_prediction_meta.shape[0]
train_prediction_meta.head()

,question_token,team_id,team_answered
0,0,45556,True
1,1,45556,True
2,2,45556,True
3,3,45556,True
4,4,45556,True


In [34]:
def e_step(model, X, meta):
    """Вычисляет матожидания ответов игроков при условии ответов команд и текущих параметров модели.
    А именно:
    1. Если команда дала верный ответ, нормирует выход модели на вероятность ответа команды 
    для каждого вопроса и для каждой команды
    2. Если команда не дала верного ответа, возвращает 0"""
    meta = meta.copy()
    pred = model.predict_proba(X)
    if len(pred.shape) == 2:
        pred = pred[:,1] if pred.shape[1] == 2 else pred.ravel()
    meta['pred'] = pred
    meta['player_fail_proba'] = 1 - pred
    team_fail_proba = meta.groupby(['question_token', 'team_id'])['player_fail_proba'].prod().rename('team_fail_proba')
    meta = meta.merge(team_fail_proba, left_on=['question_token', 'team_id'], right_index=True)
    meta['team_success_proba'] = 1 - meta['team_fail_proba']
    meta['normalized_pred'] = meta.pred / meta['team_success_proba']
    meta['expected_player_answers'] = np.where(meta.team_answered, meta.normalized_pred, 0)
    return meta.expected_player_answers.values

In [35]:
import torch

def to_tensors(X, y=None):
    assert type(X) == csr_matrix
    coo_X = X.tocoo()
    tensor_X = torch.sparse.FloatTensor(torch.LongTensor([coo_X.row.tolist(), coo_X.col.tolist()]),
                              torch.FloatTensor(coo_X.data))
    tensor_y = torch.as_tensor(y) if y is not None else None
    return tensor_X, tensor_y

In [36]:
from copy import deepcopy

class LogisticModel(torch.nn.Module):
    def __init__(self, n_features):
        super(LogisticModel, self).__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

class SoftLabelClassifier(object):
    """Классификатор, допускающий "мягкие" (из интервала [0,1]) целевые переменные """
    
    def __init__(self, coef_, intercept_, lr=1.5, epochs=25):
        n_features = coef_.shape[1]
        self.model = LogisticModel(n_features)
        self.model.linear.weight = torch.nn.parameter.Parameter(torch.as_tensor(model.coef_, dtype=torch.float32))
        self.model.linear.bias.data.fill_(intercept_[0])
        self.criterion = torch.nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, amsgrad=True)
        self.epochs = epochs
    
    def fit(self, X, y, with_tqdm=True):
        X, y = to_tensors(X, y)
        y = y.unsqueeze(1)
        epochs = range(self.epochs)
        if with_tqdm:
            epochs = tqdm(epochs, leave=True, position=0, desc='Epoch')
        for epoch in epochs:
            self.model.train()
            self.optimizer.zero_grad()
            y_pred = self.model(X)
            loss = self.criterion(y_pred, y)
            loss.backward()
            self.optimizer.step()
        return self
    
    def predict_proba(self, X):
        X, _ = to_tensors(X, None)
        self.model.eval()
        with torch.no_grad():
            return self.model(X).detach().numpy()
    
    def get_state(self):
        return deepcopy(self.model.state_dict())
    
    def set_state(self, state):
        self.model.load_state_dict(deepcopy(state))
    
    @property
    def coef_(self): 
        return self.model.linear.weight.detach().numpy()
    
    @property
    def intercept_(self): 
        return np.array([self.model.linear.bias.item()])

In [37]:
def m_step(model, X, expected_player_probas):
    model_new = model.fit(X, expected_player_probas)
    return model_new

In [38]:
def em_algo(X, y, meta, test_results, init_model=None, model=None, n_iter=10, with_tqdm=True, **kwargs):
    if model is not None:
        pass
    elif init_model is None:
        model = SoftLabelClassifier(**kwargs)
    else:
        model = SoftLabelClassifier(coef_=init_model.coef_, intercept_=init_model.intercept_, **kwargs)
    best_model_state = model.get_state()
    best_model_spearman = -1
    best_model_kendall = -1
    iterable = tqdm(range(n_iter), leave=True, position=0, desc='EM iter') if with_tqdm else range(n_iter)
    for i in iterable:
        print(f'Iteration #{i}')
        print(f'E-step')
        expected_player_probas = e_step(model, X, meta)
        print(f'M-step')
        model = m_step(model, X, expected_player_probas)
        print(f'Evaluation')
        spearman, kendall = eval_model(model, test_results, with_tqdm=False)
        if kendall > best_model_kendall:
            best_model_kendall = kendall
            best_model_spearman = spearman
            best_model_state = model.get_state()
        print(f'Spearman\'s R: {spearman}\tKendall\'s Tau: {kendall}')
        print('\n')
    model.set_state(best_model_state)
    return model, (best_model_spearman, best_model_kendall)

In [39]:
em_model, _ = em_algo(train_X, train_y, train_prediction_meta, test_results, init_model=model)

EM iter:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration #0
E-step
M-step


Epoch: 100%|██████████| 25/25 [00:53<00:00,  2.08s/it]


Evaluation


EM iter:  10%|█         | 1/10 [01:23<12:35, 83.96s/it]

...

Spearman's R: 0.7900004631012589	Kendall's Tau: 0.632005574459734


Iteration #9
E-step
M-step


Epoch: 100%|██████████| 25/25 [00:47<00:00,  1.88s/it]


Evaluation


EM iter: 100%|██████████| 10/10 [13:06<00:00, 78.61s/it]

Spearman's R: 0.7899567493286501	Kendall's Tau: 0.6319023667347948




In [40]:
spearman, kendall = eval_model(em_model, test_results)
print(f'Spearman\'s R: {spearman}\tKendall\'s Tau: {kendall}')

100%|██████████| 173/173 [00:15<00:00, 10.98it/s]

Spearman's R: 0.798509338479078	Kendall's Tau: 0.6424403022204492


Метрики подросли, но уже после двух итераций модель переобучается

## Рейтинг турниров по сложности вопросов

In [41]:
question_ratings = em_model.coef_[0, :qt.max_token+1]

In [42]:
# Перебираем все пары (id турнира, список его вопросов) и считаем среднюю сложность турнира как средний вес вопроса
# Домножаем на -1, потому что чем ниже коэффициент в модели, тем ниже шансы игроков дать верный ответ
tournament_difficulties = pd.Series({tid: -question_ratings[questions].mean()
           for tid, questions in qt.question2token.items()}, name='difficulty')

In [43]:
train_tournaments = train_tournaments.merge(tournament_difficulties, left_on='id', right_index=True)

#### Самые сложные турниры по версии модели:

In [44]:
train_tournaments.sort_values('difficulty', ascending=False).head(20)

,dateStart,id,name,questionQty,difficulty
674,2019-10-13T00:00:00+03:00,6149,Чемпионат Санкт-Петербурга. Первая лига,180,12.762155
549,2019-11-21T14:00:00+03:00,5928,Угрюмый Ёрш,45,1.531286
375,2019-06-14T19:00:00+03:00,5684,Синхрон высшей лиги Москвы,36,1.298623
650,2019-12-05T00:01:00+03:00,6101,Воображаемый музей,36,1.253255
43,2019-08-09T19:30:00+03:00,5159,Первенство правого полушария,36,1.226727
26,2019-01-04T14:00:00+03:00,5083,Ускользающая сова,36,1.130865
288,2019-04-13T12:00:00+03:00,5587,Записки охотника,36,1.125726
383,2019-08-16T19:00:00+03:00,5693,Знание – Сила VI,36,1.090563
562,2019-09-07T17:00:00+03:00,5943,Чемпионат Мира. Этап 2 Группа С,30,1.068909
226,2019-02-25T19:00:00+03:00,5515,Чемпионат Минска. Лига А. Тур четвёртый,36,1.017050


#### Самые лёгкие:

In [45]:
train_tournaments.sort_values('difficulty', ascending=True).head(20)

,dateStart,id,name,questionQty,difficulty
555,2019-10-04T19:00:00+03:00,5936,Школьная лига. I тур.,36,-3.024571
651,2019-11-15T00:00:00+03:00,6102,One ring - async,36,-3.006175
573,2019-12-06T19:00:00+03:00,5955,Школьная лига. III тур.,36,-2.986511
684,2019-10-04T19:00:00+03:00,6254,Школьная лига,216,-2.898304
572,2019-11-08T19:00:00+03:00,5954,Школьная лига. II тур.,36,-2.849718
10,2019-04-05T12:00:00+03:00,5012,Школьный Синхрон-lite. Выпуск 2.5,36,-2.800482
154,2019-02-02T00:55:00+03:00,5438,Синхрон Лиги Разума,36,-2.786644
385,2019-09-01T00:05:00+03:00,5697,Школьный Синхрон-lite. Выпуск 3.1,36,-2.775939
11,2019-04-05T12:00:00+03:00,5013,(а)Синхрон-lite. Лига старта. Эпизод V,36,-2.754029
389,2019-11-01T00:05:00+03:00,5701,Школьный Синхрон-lite. Выпуск 3.3,36,-2.684719


Выглядит вполне адекватно

## Рейтинг игроков

In [46]:
player_ratings = em_model.coef_[0, qt.max_token+1:]

In [47]:
players = train_results[['player_id', 'player_name']].drop_duplicates('player_id').reset_index(drop=True).copy()
players = players.merge(train_results.groupby('player_id')['mask_len'].sum().rename('total_questions'),
              left_on='player_id', right_index=True)
players['rating'] = player_ratings

Рейтинг по весам модели без отсечки. Действительно, большинство игроков - ноунеймы с малым числом вопросов

In [48]:
players.sort_values('rating', ascending=False).reset_index(drop=True).head(20)

,player_id,player_name,total_questions,rating
0,214337,Заворотный Юрий Александрович,36,3.748598
1,24342,Пахомов Денис Владимирович,69,3.271390
2,199961,Смирнов Владимир Александрович,36,3.184392
3,707,Александрова Елена Андреевна,48,3.121824
4,197183,Берхман Евгений Юрьевич,36,3.118120
5,173433,Кольцов Кирилл Анатольевич,35,3.091249
6,219357,Шашин Степан Андреевич,36,3.026821
7,210958,Ипполитов Денис,36,2.933783
8,191331,Чигулина Екатерина,35,2.905864
9,28015,Сальников Алексей Алексеевич,39,2.875501


Рейтинг с грубой отсечкой по числу вопросов

In [49]:
players[players.total_questions > 750].sort_values('rating', ascending=False).reset_index(drop=True).head(20)

,player_id,player_name,total_questions,rating
0,28751,Семушин Иван Николаевич,3774,2.847294
1,27403,Руссо Максим Михайлович,2178,2.842289
2,4270,Брутер Александра Владимировна,2692,2.823134
3,20691,Мереминский Станислав Григорьевич,1584,2.800493
4,27822,Савченков Михаил Владимирович,3215,2.785101
5,30152,Сорожкин Артём Сергеевич,4849,2.772407
6,30270,Спешков Сергей Леонидович,3737,2.686959
7,22935,Новиков Илья Сергеевич,1589,2.659249
8,16332,Крапиль Николай Валерьевич,1652,2.636930
9,18332,Либер Александр Витальевич,3789,2.636742


В целом выглядит хорошо. Что ещё можно сделать?
1. Выбросить игроков, которые мало играли, и переобучить модель. Не самый лучший вариант: так мы теряем информацию как о самих удалённых игроках (не сможем добавить их в рейтинг-лист или сравнить с кем-либо, корректно оценить силу команды, если такие игроки там будут), так и внесём смещения в выборку (ответы игроков зависят от ответов других членов команды)
2. Вспомнить теорему Байеса. Текущая модель не содержит априорной информации и просто оценивает силы игроков по данным. Ответил на один вопрос из одного? У нас новый чемпион! На самом же деле у нас есть представление, что случайно взятый игрок *скорее всего* будет показывать средние результаты. Как внести это знание в нашу модель? Добавить регуляризацию. Можно просто добавить L2-регуляризатор на *силы игроков*. Тогда каждый игрок получит базовый штраф, пропорциональный оценке его силы, что играет роль априорного предположения. За каждый верный или ошибочный ответ этот параметр получает обновление. Если игрок лишь несколько раз хорошо себя показал, это внесёт небольшой вклад (немного сместит априорное распределение), но если игрок регулярно показывает хорошие результаты, параметр его силы получит много обновлений, и такие байесовские свидетельства перевесят априорное распределение.

In [137]:
class SoftLabelClassifier(object):
    """Классификатор, допускающий "мягкие" (из интервала [0,1]) целевые переменные.
    Параметр reg_mask - маска регуляризации. Перед вычислением L2-нормы веса модели будут
    домножены на эту маску.
    Параметр alpha - коэффициент L2-регуляризации"""
    
    def __init__(self, coef_, intercept_, reg_mask, lr=1.5, alpha=0.0001, epochs=25):
        n_features = coef_.shape[1]
        self.model = LogisticModel(n_features)
        self.model.linear.weight = torch.nn.parameter.Parameter(torch.as_tensor(model.coef_, dtype=torch.float32))
        self.model.linear.bias.data.fill_(intercept_[0])
        self.criterion = torch.nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, amsgrad=True)
        self.epochs = epochs
        self.alpha = alpha
        self.reg_mask = reg_mask
    
    def get_reg_loss(self):
        return self.alpha * torch.norm(self.model.linear.weight * self.reg_mask, 2)
    
    def fit(self, X, y, with_tqdm=True):
        X, y = to_tensors(X, y)
        y = y.unsqueeze(1)
        epochs = range(self.epochs)
        if with_tqdm:
            epochs = tqdm(epochs, leave=True, position=0, desc='Epoch')
        for epoch in epochs:
            self.model.train()
            self.optimizer.zero_grad()
            y_pred = self.model(X)
            loss = self.criterion(y_pred, y) + self.get_reg_loss()
            loss.backward()
            self.optimizer.step()
        return self
    
    def predict_proba(self, X):
        X, _ = to_tensors(X, None)
        self.model.eval()
        with torch.no_grad():
            return self.model(X).detach().numpy()
    
    def get_state(self):
        return deepcopy(self.model.state_dict())
    
    def set_state(self, state):
        self.model.load_state_dict(deepcopy(state))
    
    @property
    def coef_(self): 
        return self.model.linear.weight.detach().numpy()
    
    @property
    def intercept_(self): 
        return np.array([self.model.linear.bias.item()])

In [127]:
reg_mask = np.zeros(train_X.shape[1], dtype=np.float32)
reg_mask[-players.shape[0]:] = 1 # Будем считать L2-норму по силам игроков
reg_mask = torch.as_tensor(reg_mask) = torch.as_tensor(reg_mask)

In [135]:
reg_em_model, _ = em_algo(train_X, train_y, train_prediction_meta, test_results,
                                   init_model=model, n_iter=2, reg_mask=reg_mask, alpha=0.0001)

EM iter:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration #0
E-step
M-step


Epoch: 100%|██████████| 25/25 [00:49<00:00,  1.89s/it]


Evaluation


EM iter:  50%|█████     | 1/2 [01:20<01:20, 80.34s/it]

Spearman's R: 0.7412081368773528	Kendall's Tau: 0.5882812094686379


Iteration #1
E-step
M-step


Epoch: 100%|██████████| 25/25 [00:47<00:00,  1.90s/it]


Evaluation


EM iter: 100%|██████████| 2/2 [02:39<00:00, 79.84s/it]

Spearman's R: 0.7180930831571098	Kendall's Tau: 0.5670585637374513




In [136]:
reg_player_ratings = reg_em_model.coef_[0, qt.max_token+1:]
reg_players = players.copy()
reg_players['rating'] = reg_player_ratings
reg_players.sort_values('rating', ascending=False).reset_index(drop=True).head(20)

,player_id,player_name,total_questions,rating
0,27403,Руссо Максим Михайлович,2178,2.954044
1,4270,Брутер Александра Владимировна,2692,2.808640
2,28751,Семушин Иван Николаевич,3774,2.799256
3,20691,Мереминский Станислав Григорьевич,1584,2.759532
4,27822,Савченков Михаил Владимирович,3215,2.758814
5,30152,Сорожкин Артём Сергеевич,4849,2.715904
6,30270,Спешков Сергей Леонидович,3737,2.694834
7,22935,Новиков Илья Сергеевич,1589,2.653643
8,22799,Николенко Сергей Игоревич,2221,2.652526
9,34328,Царёв Михаил Сергеевич,501,2.639103


Качество модели на тестовой выборке немного просело, зато рейтинг-лист выглядит довольно неплохо безо всяких отсечек.  

## Изменение рейтинга игроков со временем
Первое, что приходит в голову - это добавить каждому обучающему примеру вес, убывающий по мере отдаления даты примера от текущего момента